In [11]:
df = spark.read.csv('./data/sales.csv', header=True, inferSchema=True)

In [12]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("sales")

# Q1 Find all distinct countries.

Hint: use select(), distinct()

In [13]:
df.select("Country").distinct().show()

+-------------+
|      Country|
+-------------+
|       Sweden|
|       Jersey|
|     Malaysia|
|       Turkey|
|      Germany|
|       France|
|      Belgium|
|      Finland|
|United States|
|        India|
|       Kuwait|
|        Malta|
|        Italy|
|       Norway|
|        Spain|
|      Denmark|
|      Ireland|
|       Israel|
|      Iceland|
|  South Korea|
+-------------+
only showing top 20 rows



In [14]:
# should be same to this one
distinct_countries = spark.sql("SELECT DISTINCT Country from sales").show()

+-------------+
|      Country|
+-------------+
|       Sweden|
|       Jersey|
|     Malaysia|
|       Turkey|
|      Germany|
|       France|
|      Belgium|
|      Finland|
|United States|
|        India|
|       Kuwait|
|        Malta|
|        Italy|
|       Norway|
|        Spain|
|      Denmark|
|      Ireland|
|       Israel|
|      Iceland|
|  South Korea|
+-------------+
only showing top 20 rows



# Q2 Find the Name and Price of sales records in Brazil.

Hint: use filter().

In [17]:
df.filter(df['Country']=='Brazil').select(df['Name'], df['Price']).show()

+-------+-----+
|   Name|Price|
+-------+-----+
|Joachim| 1200|
|  Diana| 7500|
+-------+-----+



# Q3 For each country, find the total Price.

Hint: Use groupBy()

In [21]:
df.groupBy("Country").sum("Price").show()

+-------------+----------+
|      Country|sum(Price)|
+-------------+----------+
|       Sweden|      8400|
|       Jersey|      1200|
|     Malaysia|      1200|
|       Turkey|      2400|
|      Germany|     22800|
|       France|     30300|
|      Belgium|      3600|
|      Finland|      1200|
|United States|    350350|
|        India|      2400|
|       Kuwait|      1200|
|        Malta|      3600|
|        Italy|      2400|
|       Norway|     12000|
|        Spain|      2400|
|      Denmark|      8400|
|      Ireland|     29100|
|       Israel|      1200|
|      Iceland|      1200|
|  South Korea|      1200|
+-------------+----------+
only showing top 20 rows



# Q4 List countries by their total Price in descending order.

Hint: Use orderBy()

In [24]:
df.groupBy("Country").sum("Price").orderBy("Price").show()

AnalysisException: "cannot resolve '`Price`' given input columns: [Country, sum(Price)];;\n'Sort ['Price ASC NULLS FIRST], true\n+- Aggregate [Country#175], [Country#175, sum(cast(Price#170 as bigint)) AS sum(Price)#382L]\n   +- Relation[Transaction_date#168,Product#169,Price#170,Payment_Type#171,Name#172,City#173,State#174,Country#175,Account_Created#176,Last_Login#177,Latitude#178,Longitude#179] csv\n"

# Q5 Now load a second table 'countries'
http://www.cse.ust.hk/msbd5003/data/countries.csv 

df2 = spark.read.csv('countries.csv', header=True, inferSchema=True)
Redo Question 3, but replace the country names by their IDs.

Hint: Use join()

# Q6 Rewrite the PageRank example using DataFrame API.  
Here is a skeleton of the code.  Your job is to fill in the missing part.  The data files can be downloaded at:

https://www.cse.ust.hk/msbd5003/data/pagerank_data.txt

https://www.cse.ust.hk/msbd5003/data/dblp.in

In [ ]:
from pyspark.sql.functions import *

numOfIterations = 10

lines = spark.read.text("./data/pagerank_data.txt")
# You can also test your program on the follow larger data set:
# lines = spark.read.text("dblp.in")

a = lines.select(split(lines[0],' '))
links = a.select(a[0][0].alias('src'), a[0][1].alias('dst'))
outdegrees = links.groupBy('src').count()
ranks = outdegrees.select('src', lit(1).alias('rank'))

for iteration in range(numOfIterations):
# FILL IN THIS PART

ranks.orderBy(desc('rank')).show()


Note: There is a bug in the current SparkSQL implementation: The groupBy (followed by some aggregation) method sometimes fails to group all rows with the same key.  A temporary workaround is the following:

Suppose you want to do

In [ ]:
df.groupBy('A').sum('B')

If it fails to produce the desired result, try

In [ ]:
df.withColumnRenamed('A', 'A1').groupBy('A1').sum('B')

We have reported this bug to the Spark developers and the issue is currently under investigation:

https://issues.apache.org/jira/browse/SPARK-20169 (Links to an external site.)